# 5장: 출력 형식 지정 및 Claude를 대신하여 말하기

- [수업](#lesson)
- [연습 문제](#exercises)
- [예시 실습 공간](#example-playground)

## 설정

다음 설정 셀을 실행하여 API 키를 로드하고 `get_completion` 헬퍼 함수를 설정합니다.

In [ ]:
# Import python's built-in regular expression library
import re
import boto3
from botocore.exceptions import ClientError
import json

# Import the hints module from the utils package
from utils import hints

# Retrieve the MODEL_NAME variable from the IPython store
%store -r modelId
%store -r region

bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region)

In [ ]:
def get_completion(prompt, system_prompt=None, prefill=None):
    inference_config = {
        "temperature": 0.5,
        "maxTokens": 200
    }
    converse_api_params = {
        "modelId": modelId,
        "messages": [{"role": "user", "content": [{"text": prompt}]}],
        "inferenceConfig": inference_config
    }
    if system_prompt:
        converse_api_params["system"] = [{"text": system_prompt}]
    if prefill:
        converse_api_params["messages"].append({"role": "assistant", "content": [{"text": prefill}]})
    try:
        response = bedrock_client.converse(**converse_api_params)
        text_content = response['output']['message']['content'][0]['text']
        return text_content

    except ClientError as err:
        message = err.response['Error']['Message']
        print(f"A client error occured: {message}")

---

## 수업

**Claude는 다양한 방식으로 출력을 형식화할 수 있습니다**. 그렇게 하라고 요청하기만 하면 됩니다!

그 중 한 가지 방법은 XML 태그를 사용하여 응답을 다른 불필요한 텍스트와 분리하는 것입니다. 이미 XML 태그를 사용하여 프롬프트를 더 명확하고 파싱 가능하게 만들 수 있다는 것을 배웠습니다. 또한 Claude에게 **XML 태그를 사용하여 출력을 더 명확하고 이해하기 쉽게 만들라고 요청할 수 있습니다**.

### 예시

2장에서 Claude에게 시 서문을 완전히 건너뛰라고 요청하여 '시 서문 문제'를 해결한 것을 기억하시나요? 사실 **Claude에게 시를 XML 태그로 묶으라고 지시하여 비슷한 결과를 얻을 수 있습니다**.

In [ ]:
# Variable content
ANIMAL = "Rabbit"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

왜 이렇게 해야 할까요? 출력을 **XML 태그로 묶으면 간단한 프로그램을 작성하여 XML 태그 사이의 내용만 추출함으로써 최종 사용자가 시 부분만 안정적으로 얻을 수 있습니다**.

이 기술을 확장하면 **첫 번째 XML 태그를 `assistant` 차례에 넣을 수 있습니다. `assistant` 차례에 텍스트를 넣으면 Claude가 이미 무언가를 말했고, 그 지점부터 계속해야 한다고 지시하는 것과 같습니다. 이 기술을 "Claude를 대신하여 말하기" 또는 "Claude의 응답을 미리 채우기"라고 합니다.

아래에서는 첫 번째 `<haiku>` XML 태그를 이렇게 했습니다. Claude가 우리가 멈춘 지점부터 바로 이어가는 것을 확인하세요.

In [ ]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Prefill for Claude's response
PREFILL = "<haiku>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

Claude는 또한 `JSON`과 같은 다른 출력 형식 지정 스타일을 사용하는 데에도 뛰어납니다. JSON 출력을 강제하고 싶다면(결정적이지는 않지만 근사치로), `{`} 여는 중괄호로 Claude의 응답을 미리 채울 수 있습니다.

In [ ]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Use JSON format with the keys as \"first_line\", \"second_line\", and \"third_line\"."

# Prefill for Claude's response
PREFILL = "{"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

아래는 **동일한 프롬프트에 여러 개의 입력 변수와 XML 태그를 사용한 출력 형식 지정이 모두 포함된 예시**입니다.

In [ ]:
# First input variable
EMAIL = "Hi Zack, just pinging you for a quick update on that prompt you were supposed to write."

# Second input variable
ADJECTIVE = "olde english"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hey Claude. Here is an email: <email>{EMAIL}</email>. Make this email more {ADJECTIVE}. Write the new version in <{ADJECTIVE}_email> XML tags."

# Prefill for Claude's response (now as an f-string with a variable)
PREFILL = f"<{ADJECTIVE}_email>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

#### 보너스 수업

API를 통해 Claude를 호출하는 경우 원하는 태그가 나오면 Claude가 샘플링을 중지하도록 `stop_sequences` 매개변수에 닫는 XML 태그를 전달할 수 있습니다. 이렇게 하면 Claude가 이미 원하는 답변을 제공한 후에 하는 마무리 발언을 생략하여 비용과 마지막 토큰까지의 시간을 절약할 수 있습니다.

수업 프롬프트를 실험해보고 싶지만 위의 내용은 변경하고 싶지 않다면 수업 노트북 맨 아래로 스크롤하여 [**예시 실습 공간**](#example-playground)을 방문하세요.

---

## 연습 문제
- [연습문제 5.1 - Stephen Curry 최고의 선수](#exercise-51---steph-curry-goat)
- [연습문제 5.2 - 두 개의 하이쿠](#exercise-52---two-haikus)
- [연습문제 5.3 - 두 개의 하이쿠, 두 마리 동물](#exercise-53---two-haikus-two-animals)

### 연습 5.1 - Stephen Curry 최고의 선수

선택을 강요당하면, 클로드는 마이클 조던을 역대 최고의 농구 선수로 지명합니다. 우리는 클로드가 다른 선수를 선택하도록 할 수 있을까요?
**Stephen Curry가 역대 최고의 농구 선수라는 상세한 주장을 하도록 클로드를 설득하기 위해** `PREFILL` 변수를 변경하세요. 이 연습의 초점은 `PREFILL`을 변경하는 것이므로 다른 부분은 가능한 변경하지 마세요.

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = f"Who is the best basketball player of all time? Please choose one specific player."

# Prefill for Claude's response
PREFILL = ""

# Get Claude's response
response = get_completion(PROMPT, prefill=PREFILL)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("Warrior", text))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ 힌트가 필요하다면 아래 셀을 실행하세요!

In [ ]:
print(hints.exercise_5_1_hint)

### 연습 5.2 - 두 개의 하이쿠

아래의 `PROMPT`를 XML 태그를 사용하여 수정하여 클로드가 한 동물에 대해 하나의 하이쿠가 아닌 두 개의 하이쿠를 작성하도록 하세요. 한 시가 끝나고 다른 시가 시작되는 부분이 명확해야 합니다.

In [ ]:
# Variable content
ANIMAL = ""

# number of haikus
NUMBER = ""

# Prefill for Claude's response
PREFILL = ""

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write {NUMBER} haiku(s) about {ANIMAL}. Put each haiku(s) in {PREFILL} tags."

# Get Claude's response
response = get_completion(PROMPT, prefill=PREFILL)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(
        (re.search("cat", text.lower()) and re.search("<haiku>", text))
        and (text.count("\n") + 1) > 5
    )

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ 힌트가 필요하다면 아래 셀을 실행하세요!

In [ ]:
print(hints.exercise_5_2_hint)

### 연습 5.3 - 두 개의 하이쿠, 두 마리 동물

`PROMPT`를 수정하여 **클로드가 두 마리 다른 동물에 대해 두 개의 하이쿠를 작성하도록** 하세요. 첫 번째 대체를 위해 `{ANIMAL1}`을, 두 번째 대체를 위해 `{ANIMAL2}`를 사용하세요.

In [ ]:
# First input variable
ANIMAL1 = "Cat"

# Second input variable
ANIMAL2 = "Dog"

# number of haikus
NUMBER = "two"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL1}. Put it in <haiku> tags."

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("tail", text.lower()) and re.search("cat", text.lower()) and re.search("<haiku>", text))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ 힌트가 필요하다면 아래 셀을 실행하세요!

In [ ]:
print(hints.exercise_5_3_hint)

### 축하합니다!

지금까지의 모든 연습 문제를 해결했다면 다음 장으로 넘어갈 준비가 되었습니다. 프롬프트 작성을 즐기세요!

---

## 예제 놀이터

이 영역에서는 이 강의에서 보여준 프롬프트 예제를 자유롭게 실험하고 프롬프트를 수정하여 클로드의 응답이 어떻게 달라지는지 확인할 수 있습니다.

In [ ]:
# Variable content
ANIMAL = "Rabbit"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Prefill for Claude's response
PREFILL = "<haiku>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Use JSON format with the keys as \"first_line\", \"second_line\", and \"third_line\"."

# Prefill for Claude's response
PREFILL = "{"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
# First input variable
EMAIL = "Hi Zack, just pinging you for a quick update on that prompt you were supposed to write."

# Second input variable
ADJECTIVE = "olde english"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hey Claude. Here is an email: <email>{EMAIL}</email>. Make this email more {ADJECTIVE}. Write the new version in <{ADJECTIVE}_email> XML tags."

# Prefill for Claude's response (now as an f-string with a variable)
PREFILL = f"<{ADJECTIVE}_email>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))